In [ ]:
def mutateFeed(feed, prh, fs):
    
    # ADD FEEDING INFO
    feed['n'] = range(1, len(feed)+1)
    feed['nFeedEvents'] = feed['n'].max()                             # total number of feeding events
    # TIME DATA
    feed['durFP_s'] = (feed['eFPI'] - feed['sFPI'])/fs                # duration spent in feeding position
    feed['durEFP_s'] = (feed['sFPI'] - feed['sFEI'])/fs               # time it took to enter feeding position
    feed['durLFP_s'] = (feed['eFEI'] - feed['eFPI'])/fs 
    feed['hrFeed'] = np.nan
    feed['sFEDS'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S').hour) # buggy
    

    # CALC SOME DURATIONS
    for i in range(len(feed)):
        # IF THERE ARE NO START/END INDICES, GENERATE A PLOT SO THEY CAN BE SELECTED. FIRST CLICK WILL BE START, SECOND WILL BE END
        if np.isnan(feed.loc[i, 'sFPI']) == True or np.isnan(feed.loc[i, 'eFPI']) == True:
            print(i)
            sI =  feed.loc[i, 'sFEI']                                 # original prh start index
            eI =  feed.loc[i, 'eFEI']                                 # original prh end index
            rolldat = pd.DataFrame(prh['roll'][sI:eI + 1]) 
            fig, ax = plt.subplots()
            ax.plot(rolldat, label='Roll (rad)',  linewidth=2) 
            selpoints = plt.ginput(2, show_clicks=True,mouse_add=1, mouse_pop=2, mouse_stop=3)  # generate cursor
            plt.draw()
            feed.loc[i, 'sFPI'] = round(selpoints[0][0]+sI)           # start feeding position index
            feed.loc[i, 'eFPI'] = round(selpoints[1][0]+sI)           # end feeding position index
            feed.loc[i, 'durFP_s'] = (feed.loc[i, 'eFPI']-feed.loc[i, 'sFPI'])/fs # duration in feeding position (sec)
            feed.loc[i, 'durLFP_s'] = (feed.loc[i, 'eFEI']-feed.loc[i, 'eFPI'])/fs # time it too to leave feeding position (sec)
            feed.loc[i, 'durEFP_s'] = (feed.loc[i, 'sFPI']-feed.loc[i, 'sFEI'])/fs # time it took to enter feeding position
        else:
            continue
    
    for i in range(len(feed)):
        sI = feed.loc[i, 'sFPI'].astype(int)
        eI = feed.loc[i, 'eFPI'].astype(int)
        rolldat = prh['roll'][sI:eI + 1]
        #print(rolldat)
        pitchdat = prh['pitch'][sI:eI + 1]
        feed.loc[i, 'meanRollFP'] = m.degrees(np.mean(rolldat))
        feed.loc[i,'stdevRollFP'] = m.degrees(np.std(rolldat))
        feed.loc[i,'rangeRollFP'] = m.degrees(np.ptp(rolldat))
        feed.loc[i,'meanPitchFP'] = np.mean(pitchdat)
        feed.loc[i,'stdevPitchFP'] = np.std(pitchdat)
        feed.loc[i,'rangePitchFP'] = np.ptp(pitchdat)
        
    feed.head()
    return(feed)